In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np
from _utils.configurations import * #plotly configuration settings

import warnings

# Suppress Warning messages
warnings.simplefilter(action='ignore')

In [ ]:
provinces = ["Eastern Cape", "Limpopo", "Mpumalanga", "Gauteng", "Western Cape", "Northern Cape", "Hydra Central", "Kwazulu Natal", "North West", "Free State"]
provinces_corrected = ["Eastern Cape_corrected", "Limpopo_corrected", "Mpumalanga_corrected", "Gauteng_corrected", "Western Cape_corrected", "Northern Cape_corrected",
                       "Hydra Central_corrected", "Kwazulu Natal_corrected", "North West_corrected", "Free State_corrected"]
colnames = ["Date","Eastern Cape", "Limpopo", "Mpumalanga", "Gauteng", "Western Cape", "Northern Cape", "Hydra Central", "Kwazulu Natal", "North West", "Free State"]

In [ ]:
prov_2017 = pd.read_excel("data\\2017 2021 2022 Provincial loading per provincial supply area_mixed.xlsx", 
                          sheet_name = "2017_fixed", 
                          skiprows = 0, usecols = colnames, nrows = 8760,
                          index_col = "Date", parse_dates = True)
prov_2017.head()

In [ ]:
prov_2017_base = prov_2017.copy()
prov_2017_norm = prov_2017 / prov_2017.sum().sum()

In [ ]:
title = "<b>Demand in GW per Provincial Supply Area - 2017 Data (Eskom)</b><br>Individual Provincial 'Eskom Supply Area' Profiles - Not stacked"
fig = px.line(prov_2017/1_000,
              labels = {"variable":"Provincial Profile", "value":"Electricity Demand (GW)"},
              title = title,
              color_discrete_sequence=px.colors.qualitative.Prism)
fig.update_yaxes(showticklabels=True, showgrid = True, title_font = dict(size = 18))
fig.update_xaxes(showticklabels=True, showgrid = True, title_font = dict(size = 18))
fig.update_layout(legend=dict( font= dict(size=14)))
fig.update_yaxes(range=[0, 11])
fig.update_layout(
    title={ "font": dict(size=26), 'y':0.94, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
    height = 700, width = 1200, margin = dict( l=50, r=20,b=50, t=80))
fig.show()
short_title = str(title.split(">")[1].split("<")[0])
fig.write_html("figures/interactive/"+ short_title +".html")
fig.write_image("figures/static/"+ short_title +".jpg", scale = 3)

In [ ]:
eskom_2017_2023 = pd.read_csv("data\\Eskom_DataPortal_04-17_12-23_combined.csv", skiprows = 0,
                          index_col = "Date", parse_dates = True)
eskom_2017_2023["Total Generation"] = eskom_2017_2023["Dispatchable Generation"] + eskom_2017_2023["Total RE"]
eskom_2017_2023["Loadshedding"] = eskom_2017_2023["ILS Usage"] + eskom_2017_2023["Manual Load_Reduction(MLR)"] + eskom_2017_2023["IOS Excl ILS and MLR"]
#eskom_2017_2023["Total Generation_plus_loadshedding"] = eskom_2017_2023["Total Generation"] + eskom_2017_2023["Loadshedding"]
                
eskom_2017_2023.index
eskom_2019 = eskom_2017_2023["01-01-2019 00:00":"31-12-2019 23:00"]
eskom_2019.index = pd.date_range(start = "01-01-2019 00:00", end = "31-12-2019 23:00", freq = "h")
eskom_2020 = eskom_2017_2023["01-01-2020 00:00":"31-12-2020 23:00"]
eskom_2020.index = pd.date_range(start = "01-01-2020 00:00", end = "31-12-2020 23:00", freq = "h")
eskom_2021 = eskom_2017_2023["01-01-2021 00:00":"31-12-2021 23:00"]
eskom_2021.index = pd.date_range(start = "01-01-2021 00:00", end = "31-12-2021 23:00", freq = "h")
eskom_2022 = eskom_2017_2023["01-01-2022 00:00":"31-12-2022 23:00"]
eskom_2022.index = pd.date_range(start = "01-01-2022 00:00", end = "31-12-2022 23:00", freq = "h")
eskom_2023 = eskom_2017_2023["01-01-2023 00:00":"31-12-2023 23:00"]
eskom_2023.index = pd.date_range(start = "01-01-2023 00:00", end = "31-12-2023 23:00", freq = "h")

In [ ]:
model_output_2017 = pd.read_csv("data\\2017_IRPDemandREF.csv",
                                usecols = ["Date","weekday","Weighted (MW)"],
                                index_col = "Date", nrows = 8760)
model_output_2017.rename(columns = {"Weighted (MW)": "model_demand"}, inplace = True)

model_output_2017.index = pd.date_range(start = "01-01-2017 00:00:00", end = "31-12-2017 23:00:00", freq = "h")

In [ ]:
model_output_2023 = pd.read_csv("data\\IRPDemandREF.csv",
                                usecols = ["Date","weekday","Weighted (MW)"],
                                index_col = "Date", nrows = 8760)
model_output_2023.rename(columns = {"Weighted (MW)": "model_demand"}, inplace = True)

model_output_2023.index = pd.date_range(start = "01-01-2017 00:00:00", end = "31-12-2017 23:00:00", freq = "h")

In [ ]:
model_output_2050 = pd.read_csv("data\\IRPDemandREF.csv",
                                usecols = ["Date","weekday","Weighted (MW)"],
                                index_col = "Date")
model_output_2050.rename(columns = {"Weighted (MW)": "model_demand"}, inplace = True)
model_output_2050 = model_output_2050[-8760:]
model_output_2050.index = pd.date_range(start = "01-01-2017 00:00:00", end = "31-12-2017 23:00:00", freq = "h")
model_output_2050[["weekday","model_demand"]] = np.roll(model_output_2050[["weekday","model_demand"]], shift = -24, axis = 0)

In [ ]:
model_output_2050.sum()/1_000_000

In [ ]:
model_output_2050_LOW = pd.read_csv("data\\IRPDemandLOW.csv", 
                                usecols = ["Date","weekday","Weighted (MW)"],
                                index_col = "Date")
model_output_2050_LOW.rename(columns = {"Weighted (MW)": "model_demand"}, inplace = True)
model_output_2050_LOW = model_output_2050_LOW[-8760:]
model_output_2050_LOW.index = pd.date_range(start = "01-01-2017 00:00:00", end = "31-12-2017 23:00:00", freq = "h")
model_output_2050_LOW[["weekday","model_demand"]] = np.roll(model_output_2050_LOW[["weekday","model_demand"]], shift = -24, axis = 0)

In [ ]:
model_output_2050_LOW.sum()/1_000_000

In [ ]:
prov_2017 = prov_2017_norm * model_output_2017["model_demand"].sum()
prov_2017["Provincial Demand Scaled - 2017 Total"] = prov_2017.sum(axis = 1)

In [ ]:
prov_2050 = prov_2017_norm * model_output_2050["model_demand"].sum()
prov_2050["Provincial Demand Scaled - 2050 Total"] = prov_2050.sum(axis = 1)

In [ ]:
prov_2050.head()

In [ ]:
prov_2050["Provincial Demand Scaled - 2050 Total"].sum()/1_000_000

In [ ]:
historical_sentout = pd.read_excel("data\\Sentout_and_Exports_Hist_Actual.xlsx", sheet_name = "Sentout", index_col = "hour")
historical_sentout.index = pd.date_range(start = "01-01-2017 00:00:00", end = "31-12-2017 23:00:00", freq = "h")
historical_sentout.head()

In [ ]:
combined = pd.DataFrame()
combined["Provincial Demand Scaled - 2017 Total - REF"] = prov_2017["Provincial Demand Scaled - 2017 Total"]
combined["Provincial Demand Scaled - 2050 Total - REF"] = prov_2050["Provincial Demand Scaled - 2050 Total"]
combined["National Hourly Demand Model - 2017 Output - REF"] = model_output_2017["model_demand"]

combined["National Hourly Demand Model - 2023 Output - REF"] = model_output_2023["model_demand"]

combined["National Hourly Demand Model - 2050 Output - REF"] = model_output_2050["model_demand"]

combined["National Hourly Demand Model - 2050 Output - LOW"] = model_output_2050_LOW["model_demand"]

#difference_ratio = combined["National Hourly Demand Model - 2050 Output - REF"].sum() / combined["National Hourly Demand Model - 2050 Output - LOW"].sum()
#combined["National Hourly Demand Model - 2050 Output - LOW_scaled_to_REF"] = difference_ratio * combined["National Hourly Demand Model - 2050 Output - LOW"]

combined["RSA_contracted_2017"] = historical_sentout[2017]

In [ ]:
(combined.sum()/1_000_000).round(2).rename("Total Demand (TWh)").to_frame()

In [ ]:
combined["error_contracted"] = combined["Provincial Demand Scaled - 2017 Total - REF"] - combined["RSA_contracted_2017"]
combined["error_contracted_%"] = (combined["error_contracted"] / combined["RSA_contracted_2017"])*100

combined["error_modelled"] = combined["Provincial Demand Scaled - 2017 Total - REF"] - combined["National Hourly Demand Model - 2017 Output - REF"]
combined["error_modelled_%"] = (combined["error_modelled"] / combined["National Hourly Demand Model - 2017 Output - REF"])*100

In [ ]:
title = "<b>Modelled Demand Profile Comparison - REF 2017 Modelled Years</b><br>National and Total Provincial Profiles Overlaid - 2017 - Reference IRP Demand"
fig = px.line(combined[["Provincial Demand Scaled - 2017 Total - REF", "National Hourly Demand Model - 2017 Output - REF"]]/1000,
              labels = {"Date":"Date and Time", "variable":"Demand Profile Comparison", "value":"Electricity Demand (GW)"},
              title = title,
              color_discrete_sequence=px.colors.qualitative.Prism)
fig.update_yaxes(showticklabels=True, showgrid = True, title_font = dict(size = 18))
fig.update_xaxes(showticklabels=True, showgrid = True, title_font = dict(size = 18))
fig.update_layout(legend=dict(xanchor = "right", x = 0.99, y = 0.45, font= dict(size=14)))
fig.update_yaxes(range=[0, 38])
#fig.update_xaxes(range=["2017-01-01","2017-01-07"])
fig.update_layout(
    title={ "font": dict(size=26), 'y':0.94, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
    height = 700, width = 1200, margin = dict( l=50, r=20,b=50, t=80))
fig.show()
short_title = str(title.split(">")[1].split("<")[0])
fig.write_html("figures/interactive/"+ short_title +".html")
fig.write_image("figures/static/"+ short_title +".jpg", scale = 3)

In [ ]:
title = "<b>Modelled Demand Profile Comparison - Example Week - Multiple Modelled Years</b><br>National and Total Provincial Profiles Overlaid"
fig = px.line(combined[["Provincial Demand Scaled - 2017 Total - REF", "Provincial Demand Scaled - 2050 Total - REF",
                        "National Hourly Demand Model - 2017 Output - REF", "National Hourly Demand Model - 2023 Output - REF", 
                        "National Hourly Demand Model - 2050 Output - REF", "National Hourly Demand Model - 2050 Output - LOW"]]/1000,
              labels = {"Date":"Date and Time", "variable":"Demand Profile Comparison", "value":"Electricity Demand (GW)"},
              title = title,
              color_discrete_sequence=px.colors.qualitative.Prism)
fig.update_yaxes(showticklabels=True, showgrid = True, title_font = dict(size = 18))
fig.update_xaxes(showticklabels=True, showgrid = True, title_font = dict(size = 18))
fig.update_layout(legend=dict(xanchor = "right", x = 0.99, y = 0, font= dict(size=14), bgcolor='rgba(255,255,255,0.5)'))
fig.update_yaxes(range=[0, 58])
fig.update_xaxes(range=["2017-01-01","2017-01-07"])
fig.update_xaxes(range=["2017-03-13","2017-03-20"])
fig.update_layout(
    title={ "font": dict(size=26), 'y':0.94, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
    height = 700, width = 1200, margin = dict( l=50, r=20,b=50, t=80))
fig.show()
short_title = str(title.split(">")[1].split("<")[0])
fig.write_html("figures/interactive/"+ short_title +".html")
fig.write_image("figures/static/"+ short_title +".jpg", scale = 3)
#fig.write_image("figures/static/"+ short_title +".svg")

In [ ]:
title = "<b>Average Day Output Demand Profile Comparison - REF 2017</b><br>'Average Day' Profiles Compared Between Models - 2017 - Reference IRP Demand"
fig = px.line(pd.pivot_table(combined/1000,
                             index = combined.index.time, 
                             values = ["Provincial Demand Scaled - 2017 Total - REF", "National Hourly Demand Model - 2017 Output - REF"]),
              labels = {"index":"Hour of the Day", "variable":"Demand Profile Comparison", "value":"Electricity Demand (GW)"},
              title = title,
              color_discrete_sequence=px.colors.qualitative.Prism)
fig.update_yaxes(showticklabels=True, showgrid = True, title_font = dict(size = 18))
fig.update_xaxes(showticklabels=True, showgrid = True, title_font = dict(size = 18))
fig.update_layout(legend=dict(xanchor = "right", x = 0.99, y = 0.45, font= dict(size=14)))
fig.update_yaxes(range=[0, 33])
fig.update_layout(title={ "font": dict(size=26), 'y':0.94, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
    height = 700, width = 1200, margin = dict( l=50, r=20,b=50, t=80))
fig.show()
short_title = str(title.split(">")[1].split("<")[0])
fig.write_html("figures/interactive/"+ short_title +".html")
fig.write_image("figures/static/"+ short_title +".jpg", scale = 3)

In [ ]:
title = "<b>Average Day Output Demand Profile Comparison - Multiple Modelled Years</b><br>'Average Day' Profiles Compared Between Models"
fig = px.line(pd.pivot_table(combined/1000,
                             index = combined.index.time, 
                             values = ["Provincial Demand Scaled - 2017 Total - REF", "Provincial Demand Scaled - 2050 Total - REF",
                        "National Hourly Demand Model - 2017 Output - REF", "National Hourly Demand Model - 2023 Output - REF", 
                        "National Hourly Demand Model - 2050 Output - REF", "National Hourly Demand Model - 2050 Output - LOW"]),
              labels = {"index":"Hour of the Day", "variable":"Demand Profile Comparison", "value":"Electricity Demand (GW)"},
              title = title,
              color_discrete_sequence=px.colors.qualitative.Prism)
fig.update_yaxes(showticklabels=True, showgrid = True, title_font = dict(size = 18))
fig.update_xaxes(showticklabels=True, showgrid = True, title_font = dict(size = 18))
fig.update_layout(legend=dict(xanchor = "right", x = 0.99, y = 0, font= dict(size=14), bgcolor='rgba(255,255,255,0.5)'))
fig.update_yaxes(range=[0, 60])
fig.update_layout(title={ "font": dict(size=26), 'y':0.94, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
    height = 700, width = 1200, margin = dict( l=50, r=20,b=50, t=80))
fig.show()
short_title = str(title.split(">")[1].split("<")[0])
fig.write_html("figures/interactive/"+ short_title +".html")
fig.write_image("figures/static/"+ short_title +".jpg", scale = 3)

In [ ]:
title = "<b>Average Error Between Models Per Hour of Day - REF 2017</b><br>Profiles Compared Between Models - 2017 - Reference IRP Demand"
fig = px.line(pd.pivot_table(combined, index = combined.index.time, values = "error_modelled"),
        labels = {"index":"Hour of the Day", "variable":"Demand Profile Comparison", "value":"Electricity Demand (MW)"},
              title = title,
              color_discrete_sequence=px.colors.qualitative.Prism)
fig.update_yaxes(showticklabels=True, showgrid = True, title_font = dict(size = 18))
fig.update_xaxes(showticklabels=True, showgrid = True, title_font = dict(size = 18))
fig.update_layout(legend=dict( font= dict(size=14)))
fig.update_yaxes(range=[-500, 500])
fig.update_layout(
    title={ "font": dict(size=26), 'y':0.94, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
    height = 700, width = 1200, margin = dict( l=50, r=20,b=50, t=80))
fig.show()
short_title = str(title.split(">")[1].split("<")[0])
fig.write_html("figures/interactive/"+ short_title +".html")
fig.write_image("figures/static/"+ short_title +".jpg", scale = 3)

In [ ]:
title = "<b>Average Error Per Hour of Day for Each Day of Week - REF 2017</b><br>Profiles Compared Between Models - 2017 - Reference IRP Demand"
fig = px.line(pd.pivot_table(combined, index = combined.index.time, columns = combined.index.day_name(), values = "error_modelled"),
        labels = {"index":"Hour of the Day", "variable":"Demand Profile Comparison", "value":"Electricity Demand (MW)"},
              title = title,
              color_discrete_sequence=px.colors.qualitative.Prism)
fig.update_yaxes(showticklabels=True, showgrid = True, title_font = dict(size = 18))
fig.update_xaxes(showticklabels=True, showgrid = True, title_font = dict(size = 18))
fig.update_layout(legend=dict( font= dict(size=14)))
fig.update_yaxes(range=[-2100, 2000])
fig.update_layout(
    title={ "font": dict(size=26), 'y':0.94, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
    height = 700, width = 1200, margin = dict( l=50, r=20,b=50, t=80))
short_title = str(title.split(">")[1].split("<")[0])
fig.show()
fig.write_html("figures/interactive/"+ short_title +".html")
fig.write_image("figures/static/"+ short_title +".jpg", scale = 3)

In [ ]:
for province in provinces:
    prov_2050[province+"_hourly_share"] = prov_2050[province]/prov_2050["Provincial Demand Scaled - 2050 Total"]

In [ ]:
combined = prov_2050.copy()
combined["model_output_2050"] = model_output_2050["model_demand"]
combined["Provincial Demand Scaled - 2050 Total - REF"] = prov_2050["Provincial Demand Scaled - 2050 Total"]

In [ ]:
combined["error_modelled"] = combined["Provincial Demand Scaled - 2050 Total"] - combined["model_output_2050"]
combined["error_modelled_%"] = (combined["error_modelled"] / combined["model_output_2050"])*100

In [ ]:
combined[["model_output_2050","Provincial Demand Scaled - 2050 Total", "error_modelled"]].sum()/1_000_000

In [ ]:
#provincial corrections
for province in provinces:
    combined[province+"_corrected"] = combined[province] - (combined[province+"_hourly_share"] * combined["error_modelled"])

In [ ]:
combined["Provincial Demand Scaled - 2050 - REF Total_corrected"] = combined.loc[:,"Eastern Cape_corrected":"Free State_corrected"].sum(axis = 1)

In [ ]:
reduced = pd.DataFrame()
reduced["Provincial Demand Scaled - 2050 Total - REF"] = combined["Provincial Demand Scaled - 2050 Total"]
reduced["National Hourly Demand Model - 2050 Output - REF"] = combined["model_output_2050"]

In [ ]:
title = "<b>Modelled Demand Profile Comparison - REF 2050 Modelled Years</b><br>National and Total Provincial Profiles Overlaid - 2050 - Reference IRP Demand"
fig = px.line(reduced/1000, 
              height = 700, width = 1200,
              labels = {"index":"Hour of the Day", "variable":"Demand Profile Comparison", "value":"Electricity Demand (GW)"},
              title = title,
              color_discrete_sequence=px.colors.qualitative.Prism)
fig.update_yaxes(showticklabels=True, showgrid = True, title_font = dict(size = 18))
fig.update_xaxes(showticklabels=True, showgrid = True, title_font = dict(size = 18))
fig.update_layout(legend=dict(xanchor = "right", x = 0.99, y = 0.45, font= dict(size=14)))
fig.update_yaxes(range=[0, 65])
fig.update_layout(
    title={ "font": dict(size=26), 'y':0.94, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
    height = 700, width = 1200, margin = dict( l=50, r=20,b=50, t=80))
fig.show()
short_title = str(title.split(">")[1].split("<")[0])
fig.write_html("figures/interactive/"+ short_title +".html")
fig.write_image("figures/static/"+ short_title +".jpg", scale = 3)

In [ ]:
title = "<b>Modelled Demand Profile Comparison - REF 2050 Example Week</b><br>National and Total Provincial Profiles Overlaid - 2050 - Reference IRP Demand"
fig = px.line(reduced/1000, 
              height = 700, width = 1200,
              labels = {"index":"Hour of the Day", "variable":"Demand Profile Comparison", "value":"Electricity Demand (GW)"},
              title = title,
              color_discrete_sequence=px.colors.qualitative.Prism)
fig.update_yaxes(showticklabels=True, showgrid = True, title_font = dict(size = 18))
fig.update_xaxes(showticklabels=True, showgrid = True, title_font = dict(size = 18))
fig.update_layout(legend=dict(xanchor = "right", x = 0.99, y = 0.45, font= dict(size=14)))
fig.update_yaxes(range=[0, 65])
fig.update_xaxes(range=["2017-03-13","2017-03-20"])
fig.update_layout(
    title={ "font": dict(size=26), 'y':0.94, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
    height = 700, width = 1200, margin = dict( l=50, r=20,b=50, t=80))
fig.show()
short_title = str(title.split(">")[1].split("<")[0])
fig.write_html("figures/interactive/"+ short_title +".html")
fig.write_image("figures/static/"+ short_title +".jpg", scale = 3)

In [ ]:
title = "<b>Output Demand Profile Comparison (Uncorrected) - REF 2050 Average Days</b><br>'Average Day' Uncorrected Profiles Compared - 2050 - Reference IRP Demand"
fig = px.line(pd.pivot_table(reduced/1000,
                             index = combined.index.time, 
                             values = ["Provincial Demand Scaled - 2050 Total - REF", "National Hourly Demand Model - 2050 Output - REF"]),
              labels = {"index":"Hour of the Day", "variable":"Demand Profile Comparison", "value":"Electricity Demand (GW)"},
              title = title,
              color_discrete_sequence=px.colors.qualitative.Prism)
fig.update_yaxes(showticklabels=True, showgrid = True, title_font = dict(size = 18))
fig.update_xaxes(showticklabels=True, showgrid = True, title_font = dict(size = 18))
fig.update_layout(legend=dict(xanchor = "right", x = 0.99, y = 0.45, font= dict(size=14)))
fig.update_yaxes(range=[0, 65])
fig.update_layout(
    title={ "font": dict(size=26), 'y':0.94, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
    height = 700, width = 1200, margin = dict( l=50, r=20,b=50, t=80))
fig.show()
short_title = str(title.split(">")[1].split("<")[0])
fig.write_html("figures/interactive/"+ short_title +".html")
fig.write_image("figures/static/"+ short_title +".jpg", scale = 3)

In [ ]:
reduced = pd.DataFrame()
reduced["Provincial Demand Scaled - 2050 Total - REF"] = combined["Provincial Demand Scaled - 2050 Total"]
reduced["National Hourly Demand Model - 2050 Output - REF"] = combined["model_output_2050"]
reduced["error_modelled"] = combined["error_modelled"]

In [ ]:
title = "<b>Average Error Between Models Per Hour of Day - REF 2050 Average Days</b><br>Profiles Compared Between Models - 2050 - Reference IRP Demand"
fig = px.line(pd.pivot_table(reduced,
                             index = combined.index.time, 
                             values = ["error_modelled"]),
              labels = {"index":"Hour of the Day", "variable":"Demand Profile Comparison", "value":"Electricity Demand (MW)"},
              title = title,
              color_discrete_sequence=px.colors.qualitative.Prism)
fig.update_yaxes(showticklabels=True, showgrid = True, title_font = dict(size = 18))
fig.update_xaxes(showticklabels=True, showgrid = True, title_font = dict(size = 18))
fig.update_layout(legend=dict(font= dict(size=14)))
fig.update_layout(
    title={ "font": dict(size=26), 'y':0.94, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
    height = 700, width = 1200, margin = dict( l=50, r=20,b=50, t=80))
fig.show()
short_title = str(title.split(">")[1].split("<")[0])
fig.write_html("figures/interactive/"+ short_title +".html")
fig.write_image("figures/static/"+ short_title +".jpg", scale = 3)

In [ ]:
title = "<b>Average Error Per Hour of Day for Each Day of Week - REF 2050 Average Days</b><br>Profiles Compared Between Models - 2050 - Reference IRP Demand"
fig = px.line(pd.pivot_table(reduced, 
                             index = combined.index.time, 
                             columns = combined.index.day_name(), 
                             values = "error_modelled"),
              labels = {"index":"Hour of the Day", "Date":"Demand Profile Comparison", "value":"Electricity Demand (MW)"},
              title = title,
              color_discrete_sequence=px.colors.qualitative.Prism)
fig.update_yaxes(showticklabels=True, showgrid = True, title_font = dict(size = 18))
fig.update_xaxes(showticklabels=True, showgrid = True, title_font = dict(size = 18))
fig.update_layout(legend=dict(font= dict(size=14)))
fig.update_layout(
    title={ "font": dict(size=26), 'y':0.94, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
    height = 700, width = 1200, margin = dict( l=50, r=20,b=50, t=80))
fig.show()
short_title = str(title.split(">")[1].split("<")[0])
fig.write_html("figures/interactive/"+ short_title +".html")
fig.write_image("figures/static/"+ short_title +".jpg", scale = 3)

In [ ]:
reduced = pd.DataFrame()

reduced["Gauteng_corrected"] = combined["Gauteng_corrected"]
reduced["Gauteng_simple"] = combined["Gauteng"]

reduced["Western Cape_corrected"] = combined["Western Cape_corrected"]
reduced["Western Cape_simple"] = combined["Western Cape"]

reduced["Free State_corrected"] = combined["Free State_corrected"]
reduced["Free State_simple"] = combined["Free State"]

In [ ]:
title = "<b>Simple Scaling and Corrected Provincial Profiles - REF 2050 Example Week</b><br>Exemplary Provincial Profile Corrections - 2050 - Reference IRP Demand"
fig = px.line(reduced/1000, 
              labels = {"index":"Hour of the Day", "variable":"Demand Profile Comparison", "value":"Electricity Demand (GW)"},
              title = title,
              color_discrete_sequence=px.colors.qualitative.Prism)
fig.update_yaxes(showticklabels=True, showgrid = True, title_font = dict(size = 18))
fig.update_xaxes(showticklabels=True, showgrid = True, title_font = dict(size = 18))
fig.update_layout(legend=dict(xanchor = "right", x = 0.99, y = 0.45, font= dict(size=14)))
fig.update_yaxes(range=[0, 16])
fig.update_xaxes(range=["2017-03-13","2017-03-20"])
fig.update_layout(
    title={ "font": dict(size=26), 'y':0.94, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
    height = 700, width = 1200, margin = dict( l=50, r=20,b=50, t=80))
fig.show()
short_title = str(title.split(">")[1].split("<")[0])
fig.write_html("figures/interactive/"+ short_title +".html")
fig.write_image("figures/static/"+ short_title +".jpg", scale = 3)

In [ ]:
historical_exports = pd.read_excel("data\\Sentout_and_Exports_Hist_Actual.xlsx", sheet_name = "Exports", index_col = "hour")
historical_exports.index = pd.date_range(start = "01-01-2017 00:00:00", end = "31-12-2017 23:00:00", freq = "h")
historical_exports.head()

In [ ]:
historical_exports[2023]

In [ ]:
historical_exports_norm = historical_exports / historical_exports.sum()
historical_exports_norm

In [ ]:
export_weights = pd.read_csv("data\\Provincial_Export_Weights.csv", index_col = "hour")
export_weights.index = pd.date_range(start = "01-01-2017 00:00:00", end = "31-12-2017 23:00:00", freq = "h")
export_weights = export_weights.astype("float")
export_weights

In [ ]:
annual_exports = pd.read_csv("data\\Annual_Export_Scenarios_SATIM_MWh.csv", index_col = "Year")
annual_exports /= (1 - 0.039) #add 4% transmission losses from "Sales" data. 
annual_exports.head() #Values here in MWh per year

In [ ]:
historical_exports[2023]

In [ ]:
model_output_format = pd.read_csv("data\\IRPDemandREF.csv")
model_output_format["model_year"] = model_output_format.Date.values
model_output_format["model_year"] = model_output_format.model_year.str.split("/", n = 1, expand = True)[0].astype("int")
model_output_format

In [ ]:
def calculate_total_export_profile(scenario, historical_exports_norm, annual_exports, export_weights, year):
    #calculate export profiles
    montraco_hourly = 942.0091324
    if (scenario == "IRPDemandLOWDelEVs") & (year >2034):  # in the IRPDemandLOWDelEVs scenario Montracal (Mozal) export stops after 2034
        montraco_hourly = 0
    exports = pd.DataFrame()
    exports["total_export"] = historical_exports_norm[2023] * annual_exports[scenario][year]
    exports["montraco_expected"] =  montraco_hourly
    exports["montraco_actual"] = exports[["total_export","montraco_expected"]].min(axis = 1)
    exports["export_balance"] = exports["total_export"] - exports["montraco_actual"]
    
    exports_allocation = export_weights.multiply(exports["export_balance"], axis = 0)
    exports_allocation["Mpumalanga_export"] = exports_allocation["Mpumalanga_export"] + exports["montraco_actual"]
    
    return exports_allocation

In [ ]:
def provincialize_national_demand(scenario, full_ts, prov_input, output_filename, historical_exports_norm, annual_exports, export_weights, include_exports = True):    
    
    prov_shares = prov_input.copy()
    prov_shares["total"] = prov_shares.sum(axis = 1)
    
    full_ts["model_year"] = full_ts.Date.values
    full_ts["model_year"] = full_ts.model_year.str.split("/", n = 1, expand = True)[0].astype("int")
    full_ts.rename(columns = {"Weighted (MW)": "model_demand", "Overall Hour":"overall_hour"}, inplace = True)
    
    model_output_prov = full_ts.copy()
    model_output_prov["national_demand_shifted"] = 0
    model_output_prov["weekday_shifted"] = 0
    model_output_prov[provinces] = 0
    model_output_prov[provinces_corrected] = 0

    for province in provinces:
        prov_shares[province+"_hourly_share"] = prov_shares[province] / prov_shares["total"]

    results_list = []
    error_ts = pd.DataFrame()
    for year in range(full_ts.model_year.min(), full_ts.model_year.max() + 1):
        year_ts = full_ts.query("model_year == @year")
        year_ts.index = pd.date_range(start = "01-01-2017 00:00:00", end = "31-12-2017 23:00:00", freq = "h")

        modelled_start_day = year_ts.weekday.iloc[0]

        shift_hours = int(-1 * (year_ts[year_ts.weekday == 1].overall_hour.iloc[0] -1))
        year_ts[["overall_hour","weekday","model_demand","model_year"]] = np.roll(year_ts[["overall_hour","weekday","model_demand","model_year"]], shift = shift_hours, axis = 0)

        prov_year_scaled = prov_2017_norm * year_ts["model_demand"].sum()
        prov_year_scaled["total"] = prov_year_scaled.sum(axis = 1)

        error_ts = pd.DataFrame()
        error_ts["error_modelled"] = prov_year_scaled["total"] - year_ts["model_demand"]
        
        export_profile = calculate_total_export_profile(scenario = scenario, 
                                historical_exports_norm = historical_exports_norm,  
                                annual_exports = annual_exports,
                                export_weights = export_weights,
                                year = year)
        
        for province in provinces:
            #correction applied for provincial profiles based on proportional share of error compared to national total profile
            prov_year_scaled[province+"_corrected"] = prov_year_scaled[province] - (prov_shares[province+"_hourly_share"] * error_ts["error_modelled"])
            
            #add export profiles per proince
            if include_exports == True:
                prov_year_scaled[province+"_corrected"] = prov_year_scaled[province+"_corrected"] + export_profile[province+"_export"]

        #timeshift profiles back to original year    
        model_output_prov.loc[(year-2023)*8760:(1 + year-2023)*8760 -1, "national_demand_shifted"] = year_ts["model_demand"].values
        model_output_prov.loc[(year-2023)*8760:(1 + year-2023)*8760 -1 ,"weekday_shifted"] = year_ts["weekday"].values
        model_output_prov.loc[(year-2023)*8760:(1 + year-2023)*8760 -1, provinces] = prov_year_scaled[provinces].values
        model_output_prov.loc[(year-2023)*8760:(1 + year-2023)*8760 -1, provinces_corrected] = prov_year_scaled[provinces_corrected].values

        #root mean squared error metric
        rmse = np.sqrt(np.mean(error_ts**2))
        total_demand = prov_year_scaled["total"].sum()/1_000_000
        total_demand = prov_year_scaled[provinces_corrected].sum().sum()/1_000_000

        #metrics dictionary and list added per year for validation/testing
        results_dict = {"Year": year, "Total Demand": total_demand, "RSME": rmse, "Modelled Start Day": modelled_start_day}
        results_list.append(results_dict)

        print(total_demand.round(2))
        #print(rmse.round(2))
        #print("\n")

    results_stats = pd.DataFrame(results_list)
    results_stats.index = results_stats.Year
    
    fig = px.line(results_stats, height = 700)
    fig.show()
    

    model_output_prov.drop(columns = provinces, inplace = True)
    model_output_prov.to_csv("spatialized_outputs\\"+output_filename+".csv", index = False)
    
    return model_output_prov

In [ ]:
export_profile_plot_2023 = calculate_total_export_profile(scenario = "IRPDemandREF", 
                        historical_exports_norm = historical_exports_norm,  
                        annual_exports = annual_exports,
                        export_weights = export_weights,
                        year = 2023)

In [ ]:
export_profile_plot_2023 = calculate_total_export_profile(scenario = "IRPDemandREF", 
                        historical_exports_norm = historical_exports_norm,  
                        annual_exports = annual_exports,
                        export_weights = export_weights,
                        year = 2023)

title = "<b>Export Profiles Allocated and Applied per Province - 2023</b><br>2023 - Reference IRP Demand"
fig = px.line(export_profile_plot_2023, 
              labels = {"index":"Hour of the Day", "variable":"Demand Profile Comparison", "value":"Electricity Export Demand (MW)"},
              title = title,
              color_discrete_sequence=px.colors.qualitative.Prism)
fig.update_yaxes(showticklabels=True, showgrid = True, title_font = dict(size = 18))
fig.update_xaxes(showticklabels=True, showgrid = True, title_font = dict(size = 18))
fig.update_layout(legend=dict(xanchor = "right", x = 0.99, y = 0.45, font= dict(size=14)))
fig.update_yaxes(range=[0, 1200])
fig.update_xaxes(range=["2017-03-13","2017-03-20"])
fig.update_layout(
    title={ "font": dict(size=26), 'y':0.94, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
    height = 700, width = 1200, margin = dict( l=50, r=20,b=50, t=80))
fig.show()
short_title = str(title.split(">")[1].split("<")[0])
fig.write_html("figures/interactive/"+ short_title +".html")
fig.write_image("figures/static/"+ short_title +".jpg", scale = 3)

In [ ]:
export_profile_plot_2050 = calculate_total_export_profile(scenario = "IRPDemandLOWDelEVs", 
                        historical_exports_norm = historical_exports_norm,  
                        annual_exports = annual_exports,
                        export_weights = export_weights,
                        year = 2050)

title = "<b>Export Profiles Allocated and Applied per Province - 2050</b><br>2050 - IRP Low Demand with Delayed EVs (No Montraco after 2034)"
fig = px.line(export_profile_plot_2050, 
              labels = {"index":"Hour of the Day", "variable":"Demand Profile Comparison", "value":"Electricity Export Demand (MW)"},
              title = title,
              color_discrete_sequence=px.colors.qualitative.Prism)
fig.update_yaxes(showticklabels=True, showgrid = True, title_font = dict(size = 18))
fig.update_xaxes(showticklabels=True, showgrid = True, title_font = dict(size = 18))
fig.update_layout(legend=dict(xanchor = "right", x = 0.99, y = 0.45, font= dict(size=14)))
fig.update_yaxes(range=[0, 1200])
fig.update_xaxes(range=["2017-03-13","2017-03-20"])
fig.update_layout(
    title={ "font": dict(size=26), 'y':0.94, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'}, 
    height = 700, width = 1200, margin = dict( l=50, r=20,b=50, t=80))
fig.show()
short_title = str(title.split(">")[1].split("<")[0])
fig.write_html("figures/interactive/"+ short_title +".html")
fig.write_image("figures/static/"+ short_title +".jpg", scale = 3)

In [ ]:
scenario_name = "IRPDemandREF"
output1 = provincialize_national_demand(scenario = scenario_name,
                                        full_ts = pd.read_csv("data\\"+scenario_name+".csv"),
                                        prov_input = prov_2017_base,
                                        output_filename = scenario_name+"_provincial_w_exports",
                                        historical_exports_norm = historical_exports_norm,
                                        annual_exports = annual_exports,
                                        export_weights = export_weights,
                                        include_exports = True)

In [ ]:
scenario_name = "IRPDemandHIG"
output2 = provincialize_national_demand(scenario = scenario_name,
                                        full_ts = pd.read_csv("data\\"+scenario_name+".csv"),
                                        prov_input = prov_2017_base,
                                        output_filename = scenario_name+"_provincial_w_exports",
                                        historical_exports_norm = historical_exports_norm,
                                        annual_exports = annual_exports,
                                        export_weights = export_weights,
                                        include_exports = True)

In [ ]:
scenario_name = "IRPDemandLOW"
output3 = provincialize_national_demand(scenario = scenario_name,
                                        full_ts = pd.read_csv("data\\"+scenario_name+".csv"),
                                        prov_input = prov_2017_base,
                                        output_filename = scenario_name+"_provincial_w_exports",
                                        historical_exports_norm = historical_exports_norm,
                                        annual_exports = annual_exports,
                                        export_weights = export_weights,
                                        include_exports = True)

In [ ]:
scenario_name = "IRPDemandLOWDelEVs"
output4 = provincialize_national_demand(scenario = scenario_name,
                                        full_ts = pd.read_csv("data\\"+scenario_name+".csv"),
                                        prov_input = prov_2017_base,
                                        output_filename = scenario_name+"_provincial_w_exports",
                                        historical_exports_norm = historical_exports_norm,
                                        annual_exports = annual_exports,
                                        export_weights = export_weights,
                                        include_exports = True)

In [ ]:
scenario_name = "IRPDemandREF"
output1 = provincialize_national_demand(scenario = scenario_name,
                                        full_ts = pd.read_csv("data\\"+scenario_name+".csv"),
                                        prov_input = prov_2017_base,
                                        output_filename = scenario_name+"_provincial_no_exports",
                                        historical_exports_norm = historical_exports_norm,
                                        annual_exports = annual_exports,
                                        export_weights = export_weights,
                                        include_exports = False)

In [ ]:
scenario_name = "IRPDemandHIG"
output2 = provincialize_national_demand(scenario = scenario_name,
                                        full_ts = pd.read_csv("data\\"+scenario_name+".csv"),
                                        prov_input = prov_2017_base,
                                        output_filename = scenario_name+"_provincial_no_exports",
                                        historical_exports_norm = historical_exports_norm,
                                        annual_exports = annual_exports,
                                        export_weights = export_weights,
                                        include_exports = False)

In [ ]:
scenario_name = "IRPDemandLOW"
output3 = provincialize_national_demand(scenario = scenario_name,
                                        full_ts = pd.read_csv("data\\"+scenario_name+".csv"),
                                        prov_input = prov_2017_base,
                                        output_filename = scenario_name+"_provincial_no_exports",
                                        historical_exports_norm = historical_exports_norm,
                                        annual_exports = annual_exports,
                                        export_weights = export_weights,
                                        include_exports = False)

In [ ]:
scenario_name = "IRPDemandLOWDelEVs"
output4 = provincialize_national_demand(scenario = scenario_name,
                                        full_ts = pd.read_csv("data\\"+scenario_name+".csv"),
                                        prov_input = prov_2017_base,
                                        output_filename = scenario_name+"_provincial_no_exports",
                                        historical_exports_norm = historical_exports_norm,
                                        annual_exports = annual_exports,
                                        export_weights = export_weights,
                                        include_exports = False)